<a href="https://colab.research.google.com/github/HassanSherwani/Product_Purchase_Frequency/blob/master/Product_Freq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement:

How often a certain product has been sold in the past days.

#1)- Importing key modules

In [0]:
# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import sys

In [3]:
! pip install turicreate

    100% |████████████████████████████████| 87.4MB 264kB/s 
    100% |████████████████████████████████| 23.8MB 1.8MB/s 
    100% |████████████████████████████████| 2.7MB 11.5MB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
import turicreate as tc

# 2)-Loading Data

In [0]:
url = 'https://raw.githubusercontent.com/HassanSherwani/Product_Purchase_Frequency/master/20190207_transactions%20.json'

In [0]:
transactions = pd.read_json(url, lines= True)

# 3)-Exploring dataset

In [7]:
transactions.head()

,id,products
0,0,"[185, 30, 77, 188, 78, 125, 45, 155, 241, 229,..."
1,1,"[119, 148, 108, 34, 157, 82, 113, 45, 165]"
2,2,"[173, 103, 229, 240]"
3,3,[91]
4,4,"[175, 192, 54, 172]"


In [8]:
transactions.shape

(2500, 2)

In [9]:
print(transactions['products'][1415])

[250, 236, 242, 229, 92, 2, 71, 172, 109, 247, 171, 209, 90, 139, 188, 191, 145, 214, 216, 237]


In [10]:
transactions.info() # checking missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 2 columns):
id          2500 non-null int64
products    2500 non-null object
dtypes: int64(1), object(1)
memory usage: 138.6+ KB


In [11]:
transactions.describe()

,id
count,2500.00000
mean,1249.50000
std,721.83216
min,0.00000
25%,624.75000
50%,1249.50000
75%,1874.25000
max,2499.00000


# 4)- Adding Features:

### 4.1)-Create a separate dataframe for recommending users

In [0]:
customers=transactions['id']

In [13]:
customers.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [0]:
import random
random.shuffle(customers) 

 To get random values of customer ids. This would help us in avoiding sample biaseness in our modeling.

In [0]:
cust_2_rec=customers[:1000]

In [16]:
cust_2_rec.head()

0    2109
1    2383
2    2465
3    1176
4    2083
Name: id, dtype: int64

### 4.2)- break down each list of items in the products column into rows and count the number of products bought by a user

In [17]:


pd.melt(transactions.head(2).set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,id,productId,purchase_count
0,2109,30.0,1
1,2109,45.0,1
2,2109,77.0,1
3,2109,78.0,1
4,2109,89.0,2
5,2109,125.0,1
6,2109,133.0,1
7,2109,155.0,1
8,2109,161.0,1
9,2109,185.0,1


# 5- Creating features for user_id,product_id and purchase count

### 5a)-One for purchase count

In [18]:
s=time.time()

data = pd.melt(transactions.set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.01 minutes


In [19]:
data.shape

(24811, 3)

In [20]:
data.head()

,id,productId,purchase_count
0,0,10,1
1,0,15,1
2,0,43,1
3,0,133,1
4,0,169,2


### 5-b)-Normalize item

we normalize purchase frequency of each item across users by first creating a user-item matrix 

In [0]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')

In [22]:
df_matrix.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The NaN tells us that the item represented by the column was not purchased in that specific transaction.

In [23]:
df_matrix.shape

(2378, 250)

In [24]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

df_matrix_norm.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
print(df_matrix_norm.shape)

(2378, 250)


### 5-c)- create a table for input to the modeling

In [0]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()

In [27]:
data_norm.head()

,id,productId,scaled_purchase_freq
27,28,1,0.0
49,51,1,0.0
88,93,1,0.0
95,100,1,0.0
123,130,1,0.0


In [28]:
data_norm.shape

(22530, 3)

### 5-d) a function for normalizing data

In [0]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()


we have normalized the items accoreding to their purchase history, from 0–1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item.

# 6)-Split train and test set

we have three dataframes with purchase counts(data), purchase dummy(data_dummy), and scaled purchase counts(data_norm).

In [30]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(19848, 3) (4963, 3)


In [0]:
#convert dataframe to SFrame
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

A tabular, column-mutable dataframe object that can scale to big data <br>
https://turi.com/products/create/docs/generated/graphlab.SFrame.html

In [32]:
train_data

id,productId,purchase_count
1665,101,1
1233,23,1
2031,226,1
1371,231,1
1178,108,1
841,182,1
1646,224,1
1380,184,1
1762,229,1
2193,110,1


In [33]:
test_data

id,productId,purchase_count
1614,189,1
721,1,1
1398,7,1
841,245,1
1660,65,1
566,223,1
1548,28,1
2359,68,1
476,198,1
712,216,1


### 6.2)- Define a split_data function 

In [0]:
def split_data(data):

    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

### 6.3)-Apply for scaled/normalized purchase table

In [0]:
train_data_norm, test_data_norm = split_data(data_norm)

# 7)-Building Model

### 7.1)- Parameters to define field names for purchase count as target feature

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'purchase_count'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [37]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 19848 observations with 2356 users and 250 items.

Data prepared in: 0.037988s

19848 observations to process; with 250 unique items.

In [38]:
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 58578.9

+------+-----------+--------------------+------+
|  id  | productId |       score        | rank |
+------+-----------+--------------------+------+
| 2109 |    201    | 1.0909090909090908 |  1   |
| 2109 |    207    | 1.0864197530864197 |  2   |
| 2109 |    125    | 1.0789473684210527 |  3   |
| 2109 |    112    | 1.0740740740740742 |  4   |
| 2109 |     96    | 1.0731707317073171 |  5   |
| 2109 |    153    | 1.0731707317073171 |  6   |
| 2109 |     35    | 1.0684931506849316 |  7   |
| 2109 |    150    | 1.0681818181818181 |  8   |
| 2109 |    131    | 1.0675675675675675 |  9   |
| 2109 |    117    | 1.0649350649350648 |  10  |
| 2383 |    201    | 1.0909090909090908 |  1   |
| 2383 |    207    | 1.0864197530864197 |  2   |
| 2383 |    125    | 1.0789473684210527 |  3   |
| 2383 |    112    | 1.0740740740740742 |  4   |
| 2383 |     96    | 1.0731707317073171 |  5   |
| 2383 |    153    | 1.0731707317073171 |  6   |
| 2383 |     35    | 1.0684931506849316 |  7   |
| 2383 |    150    |

### Note 

Through this model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations.<br> These 30 records include 3 users and their recommended items, along with score and descending ranks.

## Checking most frequent items(as per assignment) 

In [39]:
 train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
201    1.090909
207    1.086420
125    1.078947
112    1.074074
96     1.073171
153    1.073171
35     1.068493
150    1.068182
131    1.067568
117    1.064935
156    1.064103
26     1.063291
129    1.060606
85     1.060000
101    1.058824
186    1.056338
122    1.056180
170    1.054945
247    1.054054
237    1.054054
Name: purchase_count, dtype: float64

**products 201,207,125,112,96 and 153 are the most popular (best-selling) across customers.**

### 7.2)- Applying 'scaled_purchase_freq' as target feature on model

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'scaled_purchase_freq'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [41]:
popularity_model_scaled = tc.popularity_recommender.create(train_data_norm, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 18024 observations with 2332 users and 227 items.

Data prepared in: 0.030501s

18024 observations to process; with 227 unique items.

In [42]:
popularity_recomm_4_scaled = popularity_model_scaled.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm_4_scaled.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 118133

+------+-----------+---------------------+------+
|  id  | productId |        score        | rank |
+------+-----------+---------------------+------+
| 2109 |    153    |        0.0875       |  1   |
| 2109 |    201    |       0.078125      |  2   |
| 2109 |     26    | 0.07407407407407407 |  3   |
| 2109 |    152    |  0.0707070707070707 |  4   |
| 2109 |    112    | 0.07058823529411765 |  5   |
| 2109 |     96    | 0.06666666666666667 |  6   |
| 2109 |    117    | 0.06493506493506493 |  7   |
| 2109 |    101    |        0.0625       |  8   |
| 2109 |    207    | 0.06097560975609756 |  9   |
| 2109 |     44    | 0.05813953488372093 |  10  |
| 2383 |    153    |        0.0875       |  1   |
| 2383 |    201    |       0.078125      |  2   |
| 2383 |    125    |        0.075        |  3   |
| 2383 |     26    | 0.07407407407407407 |  4   |
| 2383 |    152    |  0.0707070707070707 |  5   |
| 2383 |    112    | 0.07058823529411765 |  6   |
| 2383 |     96    | 0.06666666666666667 |  7   |


Great!.... We have all three models worked out. We have purchase frequency of each item customer-vice. <br>
Which one to trust and which one to discard? 

# 8)- Evaluate models

### 8.1)- Validation for Popularity Model on Purchase Counts

In [0]:
models_counts = [popularity_model]

In [0]:
model_names=['Popularity Model on Purchase Counts']

In [45]:
eval_counts = tc.recommender.util.compare_models(test_data, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/1931 queries. users per second: 43164.8


Precision and recall summary statistics by cutoff
+--------+----------------------+-----------------------+
| cutoff |    mean_precision    |      mean_recall      |
+--------+----------------------+-----------------------+
|   1    | 0.010875194199896425 | 0.0038852309437498425 |
|   2    | 0.011651993785603327 |  0.008543562427560358 |
|   3    | 0.012083549110996029 |  0.013067495252891411 |
|   4    | 0.012169860176074581 |  0.01802606594165374  |
|   5    | 0.011600207146556211 |  0.02156975167073567  |
|   6    | 0.010961505264975005 |  0.025237971936573667 |
|   7    | 0.010801213286971985 |  0.028643559961529903 |
|   8    | 0.011522527187985515 |  0.035389386205025714 |
|   9    | 0.011680764140629485 |  0.04093055658306823  |
|   10   | 0.011444847229414822 |  0.04428929002983901  |
+--------+----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.15247779991671445

Per User RMSE (best)
+------+------+-------+
|  id  | rmse | count |
+------+-

### 8.2)- Validation for Popularity Model on Scaled Purchase Counts

In [46]:
models_counts = [popularity_model_scaled]
model_names=['Popularity Model on Scaled Purchase Counts']
eval_counts_dummy = tc.recommender.util.compare_models(test_data_norm, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/1869 queries. users per second: 52021


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.010165864098448376 | 0.004360620652755482 |
|   2    | 0.01123595505617979  | 0.009342276236337222 |
|   3    | 0.011235955056179758 | 0.014590817600448426 |
|   4    | 0.010834670947030504 | 0.019227878417284492 |
|   5    | 0.010058855002675246 | 0.023450915946903125 |
|   6    | 0.010344212591403607 | 0.028200081530739655 |
|   7    |  0.0107009095773141  | 0.03281930749828021  |
|   8    | 0.010433386837881222 | 0.03646653417921476  |
|   9    | 0.010046965103144885 | 0.039757063874238704 |
|   10   | 0.010326377742108104 | 0.04447565543071165  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.15715670018191483

Per User RMSE (best)
+------+------+-------+
|  id  | rmse | count |
+------+------+-------+

 We tend to keep lowest rmse and it signifies least error in prediction. 



**So we will choose that as best model with rmse 0.147 i.e (in 8.1) for our final step.**

# 9) Submission

In [47]:
users_to_recommend = list(cust_2_rec)

final_model = tc.item_similarity_recommender.create(tc.SFrame(data), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_count')


Preparing data set.

Data has 24811 observations with 2378 users and 250 items.

Data prepared in: 0.037717s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 25.561ms                            | 100              | 250             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.03355s

In [48]:
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 30794.8

+------+-----------+----------------------+------+
|  id  | productId |        score         | rank |
+------+-----------+----------------------+------+
| 2109 |    168    | 0.025631868839263917 |  1   |
| 2109 |     23    | 0.022044297059377035 |  2   |
| 2109 |    169    | 0.021942555904388428 |  3   |
| 2109 |     32    | 0.02076631784439087  |  4   |
| 2109 |    191    | 0.020209228992462157 |  5   |
| 2109 |    243    | 0.019953032334645588 |  6   |
| 2109 |     81    | 0.019476604461669923 |  7   |
| 2109 |    181    | 0.019400795300801594 |  8   |
| 2109 |     17    | 0.019119056065877278 |  9   |
| 2109 |     74    | 0.01874724229176839  |  10  |
| 2383 |     79    | 0.028334187136756048 |  1   |
| 2383 |     17    | 0.027234587404463027 |  2   |
| 2383 |    149    | 0.026042726304796007 |  3   |
| 2383 |     2     | 0.02443495061662462  |  4   |
| 2383 |     40    | 0.024198671181996662 |  5   |
| 2383 |    122    | 0.02353888087802463  |  6   |
| 2383 |    191    | 0.02334354

### 9.2)- Checking most frequent items in final model

In [49]:
data.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
96     1.080808
207    1.076923
153    1.071429
201    1.069767
26     1.063158
152    1.060345
125    1.058824
112    1.058824
85     1.058824
131    1.057143
35     1.056180
63     1.054545
44     1.054545
129    1.053763
101    1.053763
117    1.052632
150    1.051724
122    1.051724
156    1.051546
120    1.050000
Name: purchase_count, dtype: float64

**RESULT: <BR>
products 96,207 , 153,201 and 26(and so on) are the most frequent selling items across customers.**

# 10)- Bonus Part- Recommending products to customers

In [50]:
# Create a csv output file
df_rec = recom.to_dataframe()
df_rec.head()

,id,productId,score,rank
0,2109,168,0.025632,1
1,2109,23,0.022044,2
2,2109,169,0.021943,3
3,2109,32,0.020766,4
4,2109,191,0.020209,5


In [51]:
print(df_rec.shape)

(10000, 4)


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['id', 'recommendedProducts']].drop_duplicates().sort_values('id').set_index('id')

In [53]:
recomendation = final_model.recommend(users=users_to_recommend, k=n_rec)

recommendations finished on 1000/1000 queries. users per second: 60716.5

In [0]:
df_rec = recomendation.to_dataframe()


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))

In [0]:

df_output = df_rec[['id', 'recommendedProducts']].drop_duplicates() \
        .sort_values('id').set_index('id')

In [57]:
df_output.head()

,recommendedProducts
id,
0,104|229|161|17|205|156|214|65|146|150
2,211|85|7|228|215|170|164|113|169|122
7,154|82|3|235|167|144|102|40|2|223
12,41|85|122|65|79|228|141|235|137|160
14,77|184|181|137|208|27|194|98|246|214


In [0]:
# define recommendation function
def customer_recomendation(id):
    if id not in df_output.index:
        print('Customer not found.')
        return id
    return df_output.loc[id]

In [59]:
customer_recomendation(48)

recommendedProducts    228|169|17|79|5|104|11|195|167|225
Name: 48, dtype: object

In [60]:
customer_recomendation(2438)

Customer not found.


2438

In [61]:
customer_recomendation(368)

recommendedProducts    211|85|7|228|215|170|164|113|169|122
Name: 368, dtype: object